Ссылка на данный блокнот: https://colab.research.google.com/drive/1Z5fVuajC1rLZQe_w2rJ2LFEzh1SV6KXq?usp=drive_link

Устанавка необходимых модулей, как и в файле "Generation_files.ipynb" для работы Spark.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

import random
import uuid
import string
import hashlib
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import Window
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Объявление схемы данных для дальнейшего считывания JSON файлов и составления Data Frame.

In [ ]:
# схема данных
schema = T.StructType([T.StructField('INN', T.StringType(), True),
                       T.StructField('raw_cookie', T.ArrayType(T.MapType(T.StringType(),T.StringType()))),
                       T.StructField('event_type', T.StringType(), True),
                       T.StructField('event_action', T.StringType(), True),
                       T.StructField('data_value', T.StringType(), True),
                       T.StructField('geocountry', T.StringType(), True),
                       T.StructField('city', T.StringType(), True),
                       T.StructField('user_os', T.StringType(), True),
                       T.StructField('systemlanguage', T.StringType(), True),
                       T.StructField('geoaltitude', T.StringType(), True),
                       T.StructField('meta_platform', T.StringType(), True),
                       T.StructField('screensize', T.StringType(), True),
                       T.StructField('timestampcolumn', T.DateType(), True)
                      ])

функция считывания файлов

In [ ]:
path_to_files = '/content/gdrive/MyDrive/data/json/'
filenames = sorted(os.listdir(path_to_files), reverse=True)

def read_file_JSON(count: int=len(filenames)-1, schema: T.StructType=schema):
  blank_DF = spark.createDataFrame([], schema=schema)
  for i in filenames[:count]:
    table = spark.read.format("json")\
                 .load(f"{path_to_files}{i}", schema=schema)
    blank_DF = blank_DF.union(table).dropDuplicates(['INN'])
  return blank_DF

In [ ]:
read_file_JSON(1).show(5)

+------------+--------------------+----------+---------------+--------------------+--------------+-----------+-------+--------------+-----------------+-------------+----------+---------------+
|         INN|          raw_cookie|event_type|   event_action|          data_value|    geocountry|       city|user_os|systemlanguage|      geoaltitude|meta_platform|screensize|timestampcolumn|
+------------+--------------------+----------+---------------+--------------------+--------------+-----------+-------+--------------+-----------------+-------------+----------+---------------+
|367351631203|[{value -> SA1.c3...|SUBMIT_MD5|       pageview|dd8d5edc996dfae5d...|      Suriname| Paramaribo|    IOS|           ENG|            4,-56|       MOBAIL| 1920x1080|     2024-02-08|
|492249402213|[{value -> SA1.39...|SUBMIT_MD5|login-check-otp|ba6292acc49001b1c...|American Samoa|  Pago Pago|Android|            RU|-14.33333333,-170|       MOBAIL| 1920x1080|     2024-02-08|
|603551334347|[{value -> SA1.a8...|

In [ ]:
DF_A = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema).drop('INN')


#df.INN <-> F.Col('INN')

In [ ]:
# схема данных
schema = T.StructType([T.StructField('INN', T.StringType(), True),
                       T.StructField('raw_cookie', T.ArrayType(T.MapType(T.StringType(),T.StringType()))),
                       T.StructField('event_type', T.StringType(), True),
                       T.StructField('event_action', T.StringType(), True),
                       T.StructField('data_value', T.StringType(), True),
                       T.StructField('geocountry', T.StringType(), True),
                       T.StructField('city', T.StringType(), True),
                       T.StructField('user_os', T.StringType(), True),
                       T.StructField('systemlanguage', T.StringType(), True),
                       T.StructField('geoaltitude', T.StringType(), True),
                       T.StructField('meta_platform', T.StringType(), True),
                       T.StructField('screensize', T.StringType(), True),
                       T.StructField('timestampcolumn', T.DateType(), True)
                      ])

# Создание витрины "В".

Согласно задания удаляем дубликаты пользователей, которые заходили с одних и тех же устройств. Для создания колонки ID, в данном случае использовалась оконная функция.


In [ ]:
# Создать цикл счтывания DF по датам. удалять дубликать на объединении дней, удалять cookie по INN
# ID
#

data_mart_B = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)
data_mart_B = data_mart_B.withColumn('raw_cookie_new',data_mart_B.raw_cookie.cast('string')).dropDuplicates(['INN']).select('INN', 'raw_cookie')
data_mart_B = data_mart_B.select(F.row_number().over(Window.orderBy(data_mart_B.INN.desc())).alias('ID_B'),
                                 'INN')
data_mart_B.show(5)

+----+------------+
|ID_B|         INN|
+----+------------+
|   1|999734191770|
|   2|999521370024|
|   3|999442171749|
|   4|998591681960|
|   5|997832375861|
+----+------------+
only showing top 5 rows



# Создание витрины "С"

Для данной и конечной витрины создана функция "search_dict", позволяющая в массиве словарей, найти необходимый словарь по значению ключа. Колонка "ID_С" сформирована с помощью втроеной монотонно увеличивающейся функции "monotonically_increasing_id" в библиотеку "pyspark.sql.functions"

In [ ]:
data_mart_C = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)

@F.udf
def search_dict(x, coockie):
  for i in x:
     if i['key'] == coockie:
      return i

data_mart_C = data_mart_C.select((F.monotonically_increasing_id()+1).alias('ID_C'),
                                  search_dict('raw_cookie', F.lit('_sa_cookie_a')).alias('_sa_cookie_a'))
data_mart_C.show(5, truncate=False)


+----+-----------------------------------------------------------------------------+
|ID_C|_sa_cookie_a                                                                 |
+----+-----------------------------------------------------------------------------+
|1   |{value=UT2.a7fbd15b-fd8b-443c-9afb-8840d10dbf51.1386391001, key=_sa_cookie_a}|
|2   |{value=OH0.a7c2ece0-8868-4011-bbaf-d09a62601ede.4074049501, key=_sa_cookie_a}|
|3   |{value=GR8.0b8aa26c-cbd1-4c67-a540-6d9086e1378f.7636108157, key=_sa_cookie_a}|
|4   |{value=NC1.5e0d3156-b981-4675-9f62-5426d601fdaa.0144777464, key=_sa_cookie_a}|
|5   |{value=ZQ3.5c86070c-ff01-4e51-a258-89ad9a553d55.0028737103, key=_sa_cookie_a}|
+----+-----------------------------------------------------------------------------+
only showing top 5 rows



In [ ]:
data_mart_C = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)

In [ ]:
data_mart_C.select(F.expr('filter(raw_cookie, x -> x.key="_sa_cookie_a")')[0]['value']).show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------+
|filter(raw_cookie, lambdafunction((namedlambdavariable()[key] = _sa_cookie_a), namedlambdavariable()))[0][value]|
+----------------------------------------------------------------------------------------------------------------+
|UT2.a7fbd15b-fd8b-443c-9afb-8840d10dbf51.1386391001                                                             |
|OH0.a7c2ece0-8868-4011-bbaf-d09a62601ede.4074049501                                                             |
|GR8.0b8aa26c-cbd1-4c67-a540-6d9086e1378f.7636108157                                                             |
|NC1.5e0d3156-b981-4675-9f62-5426d601fdaa.0144777464                                                             |
|ZQ3.5c86070c-ff01-4e51-a258-89ad9a553d55.0028737103                                                             |
+-------------------------------------------------------------------------------

# Создание витрины "D"

Тоже самое, что и в витрине "B", за исключением генерации идентификатора. Так же различия витрины "D" от "B" будет в количестве строк, при уcловии запущеного цикла в файле "Generation_files.ipynb"

In [ ]:
data_mart_D = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)
data_mart_D = data_mart_D.withColumn('raw_cookie', data_mart_D.raw_cookie.cast('string')).dropDuplicates(['INN', 'raw_cookie'])
data_mart_D = data_mart_D.select((F.monotonically_increasing_id() + 1).alias('ID_D'), 'INN')
data_mart_D.show(5, truncate=False)

+----+------------+
|ID_D|INN         |
+----+------------+
|1   |404161929124|
|2   |447616168131|
|3   |351284773140|
|4   |388097850758|
|5   |902051592399|
+----+------------+
only showing top 5 rows



# Создание витрины "E"

Создание функций "search_values_need_dict" и "transformation_values_in_md5" для нахождения значения словаря по ключу и преобразования данных в md5 соответсвенно.

In [ ]:
data_mart_E = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)

@F.udf
def search_values_need_dict(x, values_dict):
  for i in x:
    if i['key'] == values_dict:
      return i['value']

transformation_values_in_md5 = F.udf(lambda x: hashlib.md5(bytes(str(x), encoding='utf-8')).hexdigest())

data_mart_E = data_mart_E.select((F.monotonically_increasing_id() + 1).alias('ID_E'),
                                  transformation_values_in_md5(search_values_need_dict('raw_cookie',
                                  F.lit('user_phone'))).alias('hash_phone_md5'))
data_mart_E.show(5, truncate=False)

+----+--------------------------------+
|ID_E|hash_phone_md5                  |
+----+--------------------------------+
|1   |a08582397969d56c5e32daa0d99f278c|
|2   |c31a132fa921652dd6bfc0d4e70260e3|
|3   |dcdb7c0b9e82ef6cc8874fb9a3ab91c9|
|4   |f199fdbfe12df3c7ac7c94c885331616|
|5   |b33ff38bc9f82b96a48615a1bb0bc36a|
+----+--------------------------------+
only showing top 5 rows



# Создание витрины "F"

Задание схоже с витриной "F", только необходимой найти значение словоря по другому ключу. Обработка массива словарей, осуществлялась через функцию "F.explode".

In [ ]:
data_mart_F = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)

a = data_mart_F.select((F.monotonically_increasing_id() + 1).alias('ID_user'), 'raw_cookie')
a = a.select('ID_user', (F.monotonically_increasing_id() + 1).alias('ID_dict'), F.explode('raw_cookie').alias('col'))
a = a.select('ID_user', 'ID_dict',(F.monotonically_increasing_id() + 1).alias('ID'), F.explode('col').alias('keys','values'))
b = a.select('ID_user', 'ID_dict').where(F.col('values') == 'user_mail')
a = a.join(b,on=['ID_user','ID_dict']).select('ID_user', 'values').where(F.col('keys') == 'value')
data_mart_F = a.select(F.col('ID_user').alias('ID_F'), transformation_values_in_md5('values').alias('hash_email_md5'))
data_mart_F.show(5, truncate=False)

+----+--------------------------------+
|ID_F|hash_email_md5                  |
+----+--------------------------------+
|1   |8a863754fc240e058f04b236059df32d|
|2   |380636b1f9c11f29db576d06a11d0cbe|
|3   |b7c46d89b7d0598dd2d310f7c4fc0aea|
|4   |afcd705409c2dec5300f3d1c0959d822|
|5   |5b11ce54db4d9513ef41504938d01f14|
+----+--------------------------------+
only showing top 5 rows



# Создание витрины "G"

Замена значения через функцию "match_code".

In [ ]:
# pfdzprf yf Mobail
data_mart_G = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)
@F.udf
def match_code(x):
  if x == 'Mac':
    return 'IDFA'
  else:
    return 'GAID'

data_mart_G = data_mart_G.select((F.monotonically_increasing_id() + 1).alias('ID_G'),
                    search_values_need_dict('raw_cookie', F.lit('user_uid')).alias('user_uid'),
                    match_code('user_os').alias('match_code'))
data_mart_G.show(5, truncate=False)

+----+--------+----------+
|ID_G|user_uid|match_code|
+----+--------+----------+
|1   |6066449 |GAID      |
|2   |3624504 |GAID      |
|3   |6659026 |IDFA      |
|4   |2949935 |IDFA      |
|5   |8457404 |IDFA      |
+----+--------+----------+
only showing top 5 rows



# Создание обобщенной витрины объединяющая предыдущие витрины

Сначала объединяются витрина B и первоначальные данные, чтобы в дальнейшем связать другие витрины с массивом словарей.
Далее объединяюем полученую таблицу с витриной "D", чтобы дополнить список INN для этого использую "FULL JOIN".  Далее объединяем все витрины с главной с помощью "LEFT JOIN".("LEFT JOIN" используется в случает если произошла какая-то коализия данных, чтобы не произошла ситуация с пустым значением "INN")

In [ ]:
data_mart_union = spark.read.format("json").load("/content/gdrive/MyDrive/data/json/*", schema=schema)
#Объединяем с ДФ "В", так как требуется понять к какому INN какие куки принадлежат
data_mart_union = data_mart_union.join(data_mart_B, data_mart_union.INN == data_mart_B.INN, 'full').select(data_mart_B.INN,
                                                                                         data_mart_union.raw_cookie,
                                                                                         data_mart_B.ID_B,
                                                                                         data_mart_union.data_value)
#Объединяем с ДФ "D". из-за того что данных сна считывание сгенирировалось боле в данном ДФ данный будет больше, а те данные которые не объединились с ДВ "Б" будут Null
data_mart_union = data_mart_union.join(data_mart_D, data_mart_union.INN == data_mart_D.INN, 'full').select(data_mart_D.INN,
                                                                                         data_mart_union.raw_cookie,
                                                                                         data_mart_union.data_value,
                                                                                         data_mart_union.ID_B,
                                                                                         data_mart_D.ID_D
                                                                                                          )
#Создание объединённого ДФ
data_mart_union = data_mart_union.join(data_mart_E, transformation_values_in_md5(search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_phone'))) == data_mart_E.hash_phone_md5, 'left')\
               .join(data_mart_F, transformation_values_in_md5(search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_mail'))) == data_mart_F.hash_email_md5, 'left')\
               .join(data_mart_G, search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_uid')) == data_mart_G.user_uid, 'left')\
               .join(data_mart_C, search_dict(data_mart_union.raw_cookie, F.lit('_sa_cookie_a')) == data_mart_C._sa_cookie_a, 'left')\
                                 .select(data_mart_G.user_uid,
                                         data_mart_union.INN,
                                         search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_phone')).alias('user_phone'),
                                         search_values_need_dict(data_mart_union.raw_cookie, F.lit('user_mail')).alias('user_email'),
                                         data_mart_union.data_value.alias('inn_hash'),
                                         data_mart_E.hash_phone_md5,
                                         data_mart_F.hash_email_md5,
                                         search_values_need_dict(data_mart_union.raw_cookie, F.lit('org_uid')).alias('org_uid'),
                                         data_mart_union.ID_B,
                                         data_mart_union.ID_D,
                                         data_mart_E.ID_E,
                                         data_mart_F.ID_F,
                                         data_mart_G.ID_G,
                                         data_mart_C.ID_C,
                                         F.array(search_dict(data_mart_union.raw_cookie, F.lit('_sa_cookie_a')),
                                                 search_dict(data_mart_union.raw_cookie, F.lit('_fa_cookie_a')),
                                                 search_dict(data_mart_union.raw_cookie, F.lit('_ym_cookie_c')),
                                                 search_dict(data_mart_union.raw_cookie, F.lit('_fbp'))).alias('array_coockie'))
data_mart_union.show(5, truncate=False)

+--------+------------+---------------+------------------+----------------------------------------------------------------+--------------------------------+--------------------------------+-------+----+-------------+----------+----------+----------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_uid|INN         |user_phone     |user_email        |inn_hash                                                        |hash_phone_md5                  |hash_email_md5                  |org_uid|ID_B|ID_D         |ID_E      |ID_F      |ID_G      |ID_C      |array_coockie                                                                                                                                                                                                                          |
+--------+----